[Jeder kann coden](abstract/Contents.de.ipynb) / [Programmieren & TicTacToe](Programming_And_TicTacToe.de.ipynb) / [Objektorientierte Programmierung](Objectoriented_Programming.de.ipynb) / [Was sind Web APIs?](../../../WebAPIs.de.ipynb)

# Beispiel einer Spielerverwaltung als Backend

<table border="0">
  <tr>
    <td>
        <img src="">
    </td>
    <td rowspan="2">
        <a href="https://miro.com/app/board/o9J_lOJi2o0=/?moveToWidget=3458764554347680798&cot=14"><img src="../wpf/Radar_AccessingRestApi.jpg"></a>
    </td>
  </tr>
  <tr>
    <td>
      <a href="https://docs.gitlab.com/ee/api/members.html" target="_blank">GitLab API: Members</a><br>
      <a href="https://docs.gitlab.com/ee/api/" target="_blank">GitLab REST API Overview</a><br>
      <a href="https://docs.gitlab.com/ee/api/access_requests.html" target="_blank">GitLab Access Requests API</a><br>
      <a href="https://learn.microsoft.com/en-us/aspnet/core/web-api/?view=aspnetcore-8.0" target="_blank">Tutorial: Create a Web API with ASP.NET Core</a><br>
      <a href="https://learn.microsoft.com/en-us/aspnet/core/web-api/action-return-types?view=aspnetcore-8.0" target="_blank">Web API: Return Types</a><br>
      <a href="https://learn.microsoft.com/en-us/aspnet/core/fundamentals/minimal-apis?view=aspnetcore-8.0" target="_blank">Minimal APIs in ASP.NET Core</a><br>
      <a href="https://learn.microsoft.com/en-us/dotnet/api/system.net.http.httpclient?view=net-8.0" target="_blank">HttpClient Class (System.Net.Http)</a><br>
      <a href="https://learn.microsoft.com/en-us/dotnet/standard/serialization/system-text-json-overview" target="_blank">System.Text.Json – JSON in .NET</a><br>
      <a href="https://learn.microsoft.com/en-us/aspnet/core/tutorials/getting-started-with-swashbuckle?view=aspnetcore-8.0" target="_blank">Swagger/OpenAPI mit Swashbuckle</a><br>
      <a href="https://learn.microsoft.com/en-us/dotnet/csharp/programming-guide/concepts/linq/" target="_blank">Language-Integrated Query (LINQ)</a><br>
    </td>
  </tr>
</table>

Unten zeige ich dir eine vollständige Beispielimplementierung eines .NET Core WebAPI Controllers mit einer kleinen **Spielerverwaltung**, die du aus den Nutzern eines GitLab-Repositories initial befüllst. Ich erkläre die Funktionen Schritt für Schritt, damit du genau verstehst, **was im Code passiert**, und wie du das Ganze in einem Spiel nutzen kannst.

### Ziel

* GitLab-Mitglieder via REST API abrufen
* Vor- und Nachnamen trennen
* Lokal speichern (z. B. für ein Spiel als „Mitspieler“)
* Eine eigene REST API bereitstellen:

  * `GET`: Liste abrufen
  * `POST`: Spieler hinzufügen
  * `PUT`: Spieler aktualisieren
  * `DELETE`: Spieler löschen

### Projektstruktur

* `Models/Player.cs`
* `Controllers/PlayersController.cs`
* `Services/GitLabUserLoader.cs` (optional)
* `Program.cs` (API-Startpunkt)

### 1. Datenmodell `Player.cs`

```csharp
namespace GitlabPlayersAPI.Models
{
    public class Player
    {
        public int Id { get; set; }              // Eindeutige ID für die API
        public string FirstName { get; set; } = "";
        public string LastName { get; set; } = "";
    }
}
```

> Dieses Modell beschreibt einen Spieler mit Vorname, Nachname und einer internen ID. Diese ID brauchst du, um Einträge später zu updaten oder zu löschen.

### 2. Controller `PlayersController.cs`

```csharp
using Microsoft.AspNetCore.Mvc;
using GitlabPlayersAPI.Models;
using System.Collections.Generic;
using System.Linq;

namespace GitlabPlayersAPI.Controllers
{
    [ApiController]
    [Route("[controller]")]
    public class PlayersController : ControllerBase
    {
        // Interner Speicher (RAM) für Spieler
        private static readonly List<Player> Players = new();

        // Gibt alle Spieler zurück (GET /players)
        [HttpGet]
        public ActionResult<List<Player>> GetPlayers()
        {
            return Players;
        }

        // Fügt einen Spieler hinzu (POST /players)
        [HttpPost]
        public ActionResult<Player> AddPlayer([FromBody] Player newPlayer)
        {
            // Neue ID generieren
            newPlayer.Id = Players.Count > 0 ? Players.Max(p => p.Id) + 1 : 1;
            Players.Add(newPlayer);
            return CreatedAtAction(nameof(GetPlayers), new { id = newPlayer.Id }, newPlayer);
        }

        // Aktualisiert einen Spieler (PUT /players/{id})
        [HttpPut("{id}")]
        public IActionResult UpdatePlayer(int id, [FromBody] Player updated)
        {
            var existing = Players.FirstOrDefault(p => p.Id == id);
            if (existing == null) return NotFound();

            existing.FirstName = updated.FirstName;
            existing.LastName = updated.LastName;
            return NoContent();
        }

        // Löscht einen Spieler (DELETE /players/{id})
        [HttpDelete("{id}")]
        public IActionResult DeletePlayer(int id)
        {
            var player = Players.FirstOrDefault(p => p.Id == id);
            if (player == null) return NotFound();

            Players.Remove(player);
            return NoContent();
        }

        // Initialisiert aus GitLab (einmalig aufrufbar)
        [HttpPost("loadFromGitlab")]
        public async Task<IActionResult> LoadFromGitlab()
        {
            var gitlabUsers = await GitLabUserLoader.FetchUsersAsync();

            // Spieler aus GitLab-Daten erzeugen
            foreach (var user in gitlabUsers)
            {
                var parts = user.name.Split(' ', 2);
                Players.Add(new Player
                {
                    Id = user.id,
                    FirstName = parts.FirstOrDefault() ?? "",
                    LastName = parts.Skip(1).FirstOrDefault() ?? ""
                });
            }

            return Ok(Players);
        }
    }
}
```

#### Erklärung:

* `GET /players` gibt die gesamte Liste zurück.
* `POST /players` erwartet JSON-Daten (z. B. `{ "firstName": "Anna", "lastName": "Muster" }`).
* `PUT /players/2` aktualisiert z. B. den Spieler mit ID 2.
* `DELETE /players/3` entfernt den Spieler mit ID 3.
* `POST /players/loadFromGitlab` holt die Daten aus GitLab und füllt sie einmalig.

### 3. GitLab API-Aufruf `GitLabUserLoader.cs`

```csharp
using System.Net.Http;
using System.Net.Http.Headers;
using System.Text.Json;
using System.Collections.Generic;
using System.Threading.Tasks;

namespace GitlabPlayersAPI.Controllers
{
    public class GitLabUser
    {
        public int id { get; set; }
        public string name { get; set; } = "";
    }

    public static class GitLabUserLoader
    {
        public static async Task<List<GitLabUser>> FetchUsersAsync()
        {
            var client = new HttpClient();
            client.DefaultRequestHeaders.Authorization = new AuthenticationHeaderValue("Bearer", "DEIN_GITLAB_TOKEN");

            var url = "https://gitlab.example.com/api/v4/projects/PROJEKT_ID/members/all";
            var response = await client.GetAsync(url);
            response.EnsureSuccessStatusCode();

            var json = await response.Content.ReadAsStringAsync();
            var users = JsonSerializer.Deserialize<List<GitLabUser>>(json);

            return users ?? new List<GitLabUser>();
        }
    }
}
```

#### Erklärung:

* Ruft die GitLab REST API mit einem privaten Token auf.
* Nutzt das Feld `name` (z. B. „Anna Muster“) und trennt es in Vor- und Nachnamen.
* Gibt eine Liste von GitLabUsern zurück, aus der `Player`-Objekte erzeugt werden.

### 4. `Program.cs` (Startup-Datei)

```csharp
var builder = WebApplication.CreateBuilder(args);
builder.Services.AddControllers();
builder.Services.AddEndpointsApiExplorer();
builder.Services.AddSwaggerGen();

var app = builder.Build();

app.UseSwagger();
app.UseSwaggerUI();

app.UseHttpsRedirection();
app.UseAuthorization();
app.MapControllers();
app.Run();
```

> Damit bekommst du auch direkt eine Swagger UI ([http://localhost:5000/swagger](http://localhost:5000/swagger)), mit der du deine eigene REST API im Browser testen kannst.

### Beispiel-Aufruf via cURL

```bash
curl -X POST http://localhost:5000/players/loadFromGitlab
curl http://localhost:5000/players
curl -X POST http://localhost:5000/players -H "Content-Type: application/json" -d '{"firstName":"Max","lastName":"Mustermann"}'
```